# UJIAN AKHIR SEMESTER 
## TIMESERIES

### NAMA : Paul Wijaya Verda Kusuma
### NIM  : 215314051

### LSTM untuk Klasifikasi Trend Saham Evolutionary Algorithm

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

import random
from deap import base, creator, tools, algorithms

In [2]:
# ===========================
# 1. Load dan preprocess data
# ===========================
df = pd.read_csv('usd_idr_preprocessed.csv', parse_dates=['Date'], index_col='Date')
data = df['Close'].values.reshape(-1,1)

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

window_size = 20


In [3]:
# ===========================
# 2. Buat label naik/turun/stasioner
# threshold sebagai contoh (bisa di-tuning juga)
# ===========================
threshold = 0.0005  # contoh batas kecil untuk stasioner

def create_dataset_labels(dataset, window_size=20):
    X, y = [], []
    for i in range(len(dataset) - window_size - 1):
        X.append(dataset[i:i+window_size])
        diff = dataset[i+window_size+1] - dataset[i+window_size]
        if diff > threshold:
            y.append(2)  # naik
        elif diff < -threshold:
            y.append(0)  # turun
        else:
            y.append(1)  # stasioner
    return np.array(X), np.array(y)

X, y = create_dataset_labels(data_scaled, window_size)



In [4]:
# One-hot encoding untuk klasifikasi
y_cat = to_categorical(y, num_classes=3)

# Split train-test
split_idx = int(len(X)*0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y_cat[:split_idx], y_cat[split_idx:]

X_train = X_train.reshape((X_train.shape[0], window_size, 1))
X_test = X_test.reshape((X_test.shape[0], window_size, 1))

In [5]:
# ===========================
# 3. Definisi fungsi model
# ===========================
def create_model(units, optimizer_name, lr):
    model = Sequential()
    model.add(LSTM(units, activation='relu', input_shape=(window_size,1)))
    model.add(Dense(3, activation='softmax'))  # 3 kelas
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=lr)
    else:
        optimizer = RMSprop(learning_rate=lr)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [6]:
# ===========================
# 4. Definisi evaluasi fitness GA
# ===========================
def eval_fitness(individual):
    units, optimizer_idx, lr_exp, epochs = individual
    optimizer_name = ['adam', 'rmsprop'][optimizer_idx]
    lr = 10**(-lr_exp)  # log scale learning rate

    model = create_model(units, optimizer_name, lr)

    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=16,
        verbose=0,
        validation_data=(X_test, y_test)
    )

    # Pakai akurasi validasi terakhir sebagai fitness
    accuracy = history.history['val_accuracy'][-1]
    return accuracy,


In [7]:
# ===========================
# 5. Setup GA dengan DEAP
# ===========================

creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # maximize accuracy
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Parameter space
# units: 10 sampai 100
# optimizer_idx: 0 atau 1
# lr_exp: 2 sampai 5 (learning rate = 10^-lr_exp)
# epochs: 5 sampai 30

toolbox.register("units", random.randint, 10, 100)
toolbox.register("optimizer_idx", random.randint, 0, 1)
toolbox.register("lr_exp", random.randint, 2, 5)
toolbox.register("epochs", random.randint, 5, 30)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.units, toolbox.optimizer_idx, toolbox.lr_exp, toolbox.epochs), n=1)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", eval_fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=[10,0,2,5], up=[100,1,5,30], indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)


In [8]:
# ===========================
# 6. Jalankan GA
# ===========================
population = toolbox.population(n=10)
NGEN = 5
for gen in range(NGEN):
    print(f"Generation {gen+1}")

    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.2)
    fits = toolbox.map(toolbox.evaluate, offspring)

    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit

    population = toolbox.select(offspring, k=len(population))


Generation 1


C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Generation 2
Generation 3
Generation 4
Generation 5


In [9]:
# ===========================
# 7. Ambil individu terbaik
# ===========================
best_ind = tools.selBest(population, k=1)[0]
print("Best individual hyperparameters:")
print(f"Units: {best_ind[0]}, Optimizer: {'adam' if best_ind[1]==0 else 'rmsprop'}, Learning Rate: {10**(-best_ind[2])}, Epochs: {best_ind[3]}")


Best individual hyperparameters:
Units: 52, Optimizer: rmsprop, Learning Rate: 1e-05, Epochs: 17


In [10]:

# ===========================
# 8. Train ulang model terbaik dan evaluasi final
# ===========================
best_model = create_model(best_ind[0], ['adam','rmsprop'][best_ind[1]], 10**(-best_ind[2]))
history = best_model.fit(X_train, y_train, epochs=best_ind[3], batch_size=16, verbose=1)

score = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Final Test Loss: {score[0]}, Test Accuracy: {score[1]}")


Epoch 1/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3526 - loss: 1.0986 
Epoch 2/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3514 - loss: 1.0982 
Epoch 3/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3304 - loss: 1.0986 
Epoch 4/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3617 - loss: 1.0981 
Epoch 5/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3359 - loss: 1.0983   
Epoch 6/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3480 - loss: 1.0982 
Epoch 7/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3455 - loss: 1.0979 
Epoch 8/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3533 - loss: 1.0978 
Epoch 9/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3487 - loss: 1.0975   
Epoch 10/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3442 - loss: 1.0978 
Epoch 11/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3522 - loss: 1.0970 
Epoch 12/17
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accura